In [4]:
import torch

cudaIf = torch.cuda.is_available()
device = torch.device("cuda" if cudaIf else "cpu")
print("device=", device)


device= cuda


In [1]:

# 创建参数对象
from src.optionsClass import TrainingOptions

# Create an instance of TrainingOptions
opt = TrainingOptions()

# Set the values as per the shell commands
opt.data_dir = "./data"
opt.log_dir = "./logs"
opt.coeff_fpath = "chebychev"
opt.depth_flat_world_fpath = "depth_flat_world"
opt.model_name = "multinetwork"
opt.model_type = "multinetwork"
opt.exp_name = "multinetwork"
opt.models_to_load = ["depth", "ambient", "encoder", "albedo", "pose", "pose_encoder"]
opt.load_weights_folder = "./models/initialization"
opt.exp_num = 0
opt.height = 512
opt.width = 1024
opt.num_bits = 10
opt.scales = [0]
opt.frame_ids = [0, -1, 1]
opt.pose_model_type = "separate_resnet"
opt.num_layers = 18
opt.weights_init = "pretrained"
opt.pose_model_input = "pairs"
opt.min_depth = 0.1
opt.max_depth = 100.0
opt.dataset = "gated"
opt.split = "gated2gated"
opt.batch_size = 4
opt.num_workers = 4
opt.learning_rate = 2e-4
opt.num_epochs = 20
opt.scheduler_step_size = 15
opt.disparity_smoothness = 0.001
opt.log_frequency = 200
opt.save_frequency = 1
opt.cycle_weight = 0.05
opt.depth_normalizer = 70.0
opt.passive_weight = 0.01
opt.cycle_loss = True
opt.temporal_loss = True
opt.sim_gated = True
opt.v1_multiscale = True
opt.infty_hole_mask = True
opt.snr_mask = True
opt.intensity_mask = True
opt.passive_supervision = True


In [3]:
from src.trainer import Trainer

trainer = Trainer(opt)
trainer.train()

AssertionError: Cannot find folder ./models/initialization